In [70]:
import nltk
import os
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import TweetTokenizer
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import json
import pickle
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from spellchecker import SpellChecker
import enchant 
from tensorflow.python.framework import ops
import tflearn
nltk.download('stopwords')
stoplist = set(stopwords.words("english"))
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\60122\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\60122\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [71]:
#read the data
data = pd.read_csv(r"D:\Study Material\FYP\dataset.csv")

In [72]:
#filter the intent
data = data[['cancel_order', 'check_refund_policy', 'complaint', 'contact_customer_service', 'contact_human_agent', 'delete_account', 'delivery_options', 'delivery_period', 'edit_account', 'get_refund', 'recover_password','review']]

In [73]:
# add intent to the list 

intents = []
for num,intent in enumerate(data):
    intents.append(intent)

print(intents)

['cancel_order', 'check_refund_policy', 'complaint', 'contact_customer_service', 'contact_human_agent', 'delete_account', 'delivery_options', 'delivery_period', 'edit_account', 'get_refund', 'recover_password', 'review']


In [74]:
# create three lists to store corpus and intent
docs_x = []
docs_y = []
# perform tokenization using Regex (w+ matches any words except newline)
tokenizer =  TweetTokenizer()
#loop through dataset
for column in data:
    
    for row in data[column]:
        #convert r to string and lowercase it 
        
        r = str(row)
        r = r.lower()
        # if r is nan , skip it , otherwise add both data and intent into list 
        if r != 'nan' :
            tokens = tokenizer.tokenize(r)
            #nltk.word_tokenize(r)
            #add each tokens to list 
            docs_x.extend(tokens)
            docs_y.append(column)
        else:
            continue


In [75]:
#create an enchant dict
eng_dict = enchant.Dict("en_US")
#create a stopwords set from nltk
stop_words = set(stopwords.words('english')) 
spell = SpellChecker()
#create a list to store tokens without stopwords
filtered_sentence = [] 
  
# if the words is not same as any stopwords, add into list
for w in docs_x: 
    w = str(w)
    if w not in stop_words: 
         filtered_sentence.append(w) 

  


In [76]:
# store in a list called token 
token = sorted(set(filtered_sentence))

In [77]:

import pkg_resources
from symspellpy.symspellpy import SymSpell


unique_list = dict()
# Set max_dictionary_edit_distance to avoid spelling correction
sym_spell = SymSpell(max_dictionary_edit_distance=1, prefix_length=2)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# a sentence without any spaces
for w in token:
    input_term = w
    result = sym_spell.word_segmentation(input_term)
    unique_list[input_term] = result.corrected_string
    #print("{}:{}, {}, {}".format(w,result.corrected_string, result.distance_sum,
                          #result.log_prob_sum))
   

In [78]:
#print(len(unique_list))

In [79]:

for key in unique_list.copy():
    if eng_dict.check(key) == True:
        print("{}:{}".format(key,unique_list[key]))
        unique_list.pop(key)
# from nltk.corpus import words
# nltk.download('words')
# for key in unique_list:
#     if key in set(words.words()) == True:
#         print("{}:{}".format(key,unique_list[key]))

#print(len(unique_list))

account:account
account's:accounts
address:address
agent:agent
ane:ane
ant:ant
anything:anything
ar:a
arrive:arrive
ask:ask
available:available
ave:ave
back:back
ben:ben
bet:bet
ca:a
call:call
calling:calling
cam:cam
can't:can't
cancel:cancel
canceled:cancelled
canceling:cancelling
cancelled:cancelled
cancelling:cancelling
cannot:can not
cant:cant
case:case
cat:cat
change:change
changed:changed
changes:changes
changing:changing
check:check
checking:checking
client:client
close:close
closing:closing
comment:comment
complaint:complaint
concert:concert
consumer:consumer
contact:contact
contacting:contacting
could:could
customer:customer
delete:delete
deleting:deleting
delivery:delivery
discontent:discontent
dot:dot
edit:edit
editing:editing
ell:ell
em:pm
email:email
event:event
expect:expect
fast:fast
fees:fees
fie:fie
file:file
filing:filing
find:find
fir:fir
fo:of
fora:for a
forgot:forgot
free:free
fucking:fucking
full:full
game:game
get:get
getting:getting
give:give
go:go
going:going
g

In [80]:
def add_values_in_dict(sample_dict, key, list_of_values):
    """Append multiple values to a key in the given dictionary"""
    if key not in sample_dict:
        sample_dict[key] = list()
    sample_dict[key].extend(list_of_values)
    return sample_dict

In [81]:
#create another dataset
#form a dictonary to store the words
#replace the old words with new words
#perform stemming/lemma
new_list = dict()
storing = []

import math

for intent in data:
    for sen in data[intent]:
        sen = str(sen)
        sen = sen.lower()
        if sen != 'nan':
            nltk_tokens = tokenizer.tokenize(sen)
            for word in nltk_tokens:
                for count,key in enumerate(unique_list.keys()):
                    if word not in storing and word == key:
                        #print("{}:{}".format(word,key))
                        storing.extend([unique_list[key]])
                        #storing.extend(word.replace(key, unique_list[key]))
                        break
                    elif word not in storing and word != key and count >= len(unique_list) - 1:
                        storing.extend([word])
                        #storing.extend(word.replace(key, unique_list[key]))
                    else:    
                        continue
            new_list = add_values_in_dict(new_list,intent,[storing])
            #print(new_list)
            storing = []
        else:
             continue
                


In [82]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

filtered = {}

for intent in new_list:
    for sentence in new_list[intent]:
        #TreebankWordDetokenizer().detokenize(sentence)
        filtered = add_values_in_dict(filtered,intent,[TreebankWordDetokenizer().detokenize(sentence)])

filtered = pd.DataFrame((dict([ (k,pd.Series(v)) for k,v in filtered.items()])))
filtered

,cancel_order,check_refund_policy,complaint,contact_customer_service,contact_human_agent,delete_account,delivery_options,delivery_period,edit_account,get_refund,recover_password,review
0,help me cancel an order,where can i see your refund policy a,i'm not happy with the service a what do i hav...,how to call customer support a please,how do i talk to a person a,how i could remove my account a please,how to view the options for ticket delivery,can you give me information about the delivery...,how to update my profile a,i wanna obtain a reimbursement a what should do a,i lost my account password a how can retrieve ...,i need help leaving an opinion
1,i want to cancel a purchase a what do have a,i want to check your refund policy a what can ...,i am discontent with the service a how could f...,i have a question a what do to call customer s...,how to talk a human a please a,i changed my mind a can you delete use account a,how to check the delivery options,how fast can i expect my ticket to arrive a,help me modify the information on my profile,the show was postponed a what do i have to obt...,please a to retrieved my password a,how can i write an opinion for a service a
2,how do i cancel an order a,where do i view your money back guarantee a,i am not happy with the service a could file c...,can you give me information about the email of...,i can't understand you a how do contact human a,i changed my mind a how do cancel account a,i want information about the ticket delivery o...,i want to know how soon can expect my ticket a...,how to edit the information on my profile,the show was postponed a how do i request refu...,i lost my password a how can reset it a,i am happy with the service and wanna lodge an...
3,can i cancel an order a,i want to check your money back policy a what ...,in discontent with the service and i want to f...,can you show me the email of customer service a,i can't understand you a contact human a please,i don't want my user account a what should do ...,can you give me information about the options ...,i want information about the delivery period f...,can i modify the information on my profile a,the game was cancelled a get refund,i want to reset my account password a what do ...,i'm happy with the service a how do i submit a...
4,i wanna cancel an order,how can i see your money back policy a,i want to submit a complaint a what do have a,can you give me information about the email of...,i need help talking to a human agent,i done want my online account and wanna remove it,i wanna see the options for delivery a what sh...,i want to know when my tickets are going arrive,help me update my profile,the event was postponed and i wanna obtain a r...,i forgot my password a retrieve it a please,i want to submit a review a what do have a
...,...,...,...,...,...,...,...,...,...,...,...,...
2050,NaN,NaN,NaN,i want information a can you speak with custom...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2051,NaN,NaN,NaN,can you tell me what the email address of clie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2052,NaN,NaN,NaN,i want information a how can send an email to ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2053,NaN,NaN,NaN,can you tell me the email address to contact c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
# create three lists to store corpus and intent
tokens = []
words = []
docs_x = []
docs_y = []
# perform tokenization using Regex (w+ matches any words except newline)
tokenizer =  TweetTokenizer()
#loop through dataset
for column in filtered:
    
    for row in filtered[column]:
        #convert r to string and lowercase it 
        r = str(row)
        # if r is nan , skip it , otherwise add both data and intent into list 
        if r != 'nan' :
            tokens = tokenizer.tokenize(r)
            #add each tokens to list 
            words.extend(tokens)
            docs_x.append(tokens)
            docs_y.append(column)
        else:
            continue
            

In [84]:

def stem_or_lemma(word,vector):
    if word == "snow":
        
        stemmer = SnowballStemmer(language = 'english', ignore_stopwords=True)
    elif word == "porter":
        
        stemmer = PorterStemmer()
    elif word == "lan":
        
        stemmer = LancasterStemmer()     
    words = [stemmer.stem(w.lower()) for w in vector if w not in stoplist]
    words = sorted(list(set(words)))
    
    return words

In [85]:
print(stem_or_lemma("snow",words))

['account', 'address', 'agent', 'alex', 'ane', 'ant', 'anyth', 'ar', 'arn', 'arrest', 'arriv', 'ask', 'avail', 'ave', 'back', 'ben', 'bet', 'ca', 'call', 'cam', "can't", 'cancel', 'cannot', 'cant', 'cart', 'case', 'cat', 'chang', 'check', 'client', 'close', 'closest', 'co', 'comment', 'complaint', 'concert', 'consum', 'contact', 'content', 'could', 'custom', 'defil', 'delet', 'deliveri', 'discont', 'done', 'donor', 'dot', 'edg', 'edit', 'ell', 'em', 'email', 'event', 'expect', 'fast', 'fee', 'fie', 'file', 'find', 'fir', 'fo', 'fora', 'forgot', 'form', 'free', 'fuck', 'full', 'fund', 'game', 'get', 'give', 'go', 'gonna', 'guarante', 'hack', 'happi', 'hat', 'help', 'hep', 'ho', 'hos', 'hour', 'howdi', 'human', "i'm", 'id', 'ii', 'ike', 'inc', 'inform', 'int', 'interest', 'king', 'kn', 'know', 'leas', 'leav', 'like', 'lode', 'lodg', 'long', 'lost', 'lot', 'mail', 'make', 'man', 'min', 'mind', 'modifi', 'money', 'mouser', 'mt', 'n', 'nake', 'ned', 'need', 'nit', 'number', 'obtain', 'one',

In [86]:
training = []
output = []

out_empty = [0 for _ in range(len(intents))]
for x,doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[intents.index(docs_y[x])] = 1
    training.append(bag)
    output.append(output_row)

In [87]:
training = np.array(training)
output = np.array(output)
training.shape
output.shape

(8663, 12)

In [88]:
print(training)

[[1 1 1 ... 0 0 0]
 [0 0 1 ... 0 0 1]
 [0 0 1 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 1 1]]


In [89]:
print(output)

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


In [90]:
#Saving data to disk
with open("snowdata.pickle","wb") as f:
    pickle.dump((words, labels, training, output),f)
 


In [91]:
infile = open("D:\Study Material\FYP\snowdata.pickle",'rb')
new_dict = pickle.load(infile)
infile.close()

In [36]:
print(new_dict)

(['account', "account's", 'address', 'ag', 'alex', 'an', 'ant', 'anyth', 'ar', 'arn', 'arrest', 'ask', 'av', 'avail', 'back', 'ben', 'bet', 'ca', 'cal', 'cam', "can't", 'cancel', 'cannot', 'cant', 'cart', 'cas', 'cat', 'chang', 'check', 'cli', 'clos', 'closest', 'co', 'com', 'complaint', 'concert', 'consum', 'cont', 'contact', 'could', 'custom', 'defil', 'delet', 'delivery', 'discont', 'don', 'dot', 'edg', 'edit', 'el', 'em', 'email', 'ev', 'expect', 'fast', 'fee', 'fie', 'fil', 'find', 'fir', 'fo', 'for', 'forgot', 'form', 'fre', 'fuck', 'ful', 'fund', 'gam', 'get', 'giv', 'go', 'going', 'gonn', 'guar', 'hack', 'happy', 'hat', 'help', 'hep', 'ho', 'hos', 'hour', 'howdy', 'hum', "i'm", 'id', 'ii', 'ik', 'inform', 'int', 'interest', 'king', 'kn', 'know', 'leas', 'leav', 'lik', 'lod', 'lodg', 'long', 'lost', 'lot', 'mail', 'mak', 'man', 'min', 'mind', 'mod', 'money', 'mous', 'mt', 'n', 'nak', 'ned', 'nee', 'nit', 'numb', 'obtain', 'on', 'onlin', 'ont', 'op', 'opin', 'opt', 'ord', 'passwo

In [92]:

ops.reset_default_graph()
net = tflearn.input_data(shape = [None, len(training[0])])
net = tflearn.fully_connected(net,20)
net = tflearn.fully_connected(net,20)
net = tflearn.fully_connected(net,len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch = 20, batch_size = 8, show_metric = True)
model.save("snowmodel.tflearn")

Training Step: 21659  | total loss: 0.70532 | time: 8.945s
| Adam | epoch: 020 | loss: 0.70532 - acc: 0.9475 -- iter: 8656/8663
Training Step: 21660  | total loss: 0.63479 | time: 8.953s
| Adam | epoch: 020 | loss: 0.63479 - acc: 0.9527 -- iter: 8663/8663
--
INFO:tensorflow:D:\Study Material\FYP\snowmodel.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [35]:
df = pd.DataFrame({'Sentence':training,'Label':output})
df

ValueError: If using all scalar values, you must pass an index